# 1) Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

**Ответ:**

- Hit rate@k = был ли хотя бы 1 релевантный товар среди топ-k рекомендованных

- Precision@k = доля купленных релевантных товаров среди топ-k рекомендованных

В общих задачах предпочтительнее использовать Precision@k, т.к. это более более точна метрика качества выборки предлагаемых товаров.

Метрику precision@k необходимо применять в том случае, если покупатель нам известен и мы значем его предпочтения

Метрику hit_rate@k можно использовать для распределения пользователей по группам интересов. Также использутся при продаже дорогих товаров

Пример:

- Рекомендация музыки на яндексмузыке. Предпочтительней precision@k. музыка - товар, юзер - клиент.

- продажа техники: техника - товар, юзер - клиент.


# 2)В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

**Ответ:**

Использование логарифма в метриках ранжирования используются для того, чтобы влияние номера позиции товара оказывало максимальное значение на выдачу. Цель алгоритма - правильно ранжировать первые k объектов, все позиции. В случае выдачи Яндекса между позициями 1 и 11 целая пропасть (лишь в нескольких случаях из ста пользователь заходит дальшей первой страницы поисковой выдачи), а между позициями 101 и 111 особой разницы нет — до них мало кто доходит. Эти рассуждения выражаются с помощью логарифма

Вместо логарифма можно выбрать любую медленно возрастающую функцию, напрмер корень из x.

Discounted cumulative gain at K (DCG@K) и normalized discounted cumulative gain at K (nDCG@K)

# 3)Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).

Ранговый коэффициент корреляции Кендэлла, который основан на подсчете согласованных (и несогласованных) пар у перестановок — пар элементов, котором перестановки присвоили одинаковый (разный) порядок.

Cumulative gain at K (CG@K) — базовая метрика ранжирования, которая использует простую идею: чем релевантные элементы в этом топе, тем лучше.

Expected reciprocal rank (ERR) — пример метрики качества ранжирования, основанной на каскадной модели.


In [196]:
import pandas as pd
import numpy as np
from collections import Counter

In [197]:
# pr@8 - точность на K элементах

def pr_k(recommended_list, bought_list, k=8): # точность на K элементах
    
    bought_list = np.array(bought_list) #преобразуем списки в массив
    recommended_list = np.array(recommended_list)[:k] #преобразуем в массив и возьмём первые k рекомендованных товаров
    
    flags = np.isin(bought_list, recommended_list) # выбираем купленные товары из рекомендованных
  
    precision = flags.sum() / len(recommended_list) # вычисляеи процент купленных товаров из первых k рекомендованных
    
    
    return precision

# rec@8  полнота на K элементах


def rec_k(recommended_list, bought_list, k=8):
    bought_list = np.array(bought_list) # преобразуем список в массив
    recommended_list = np.array(recommended_list)[:k] #преобразуем в массив и возьмём первые k рекомендованных товаров
    
    flags = np.isin(bought_list, recommended_list) #выбираем купленные товары из рекомендованных
    
    recall = flags.sum() / len(bought_list) # вычисляем rec@k
    
    return recall
  
# AP@8 Average precision at K


def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = pr_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

# NDCG@8 Normalized discounted cumulative gain

def NDCG_k(recommended_list, bought_list, k=8):
  
 
  recommended_list = np.array(recommended_list)
  
  
  kDCG_k=bought_list.count(recommended_list[0]); # вычисляем начальное условие 
  
  t=1

  
  for i in recommended_list[1:k]: # вычисляем количество покупок из рекомендованных (с учетом десятичных логарифмов)
    t+=1
    n = bought_list.count(i)
    kDCG_k+=n/np.log10(t)

  t=1
  kiDCG_k=1

  for i in range(1,k):  # вычичляем идеальное состояние
    t+=1
    kiDCG_k=kiDCG_k+1/np.log10(t)

  
  return kDCG_k/kiDCG_k # возвращаем искомую метрику NDCG


# Не очень понятно какая именно метрика имелась в виду, поэтому приведу 2 метики: RR@8 Retention Rate и RR@k Reciprocal Rank


# RR@8 Количество купленных товаров / Общее число предложенных из k предложенных

def RR_k(recommended_list, bought_list, k=8):
   n=0
   for i in recommended_list[:k]: # вычисляем количество покупок из рекомендованных (с учетом десятичных логарифмов)
      n+=bought_list.count(i)
   
   return n/k


# Reciprocal Rank

def reciprocal_rank(recommended_list, bought_list, k):
    ranks=0.
    for item_rec in recommended_list[:k]:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)
  
# ERR@8


def expected_reciprocal_rank(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    item_relevant = [(recommended_list[i] in bought_list) for i in range(recommended_list.shape[0])]
   
    rank_item_relevant = [i+1 for i in range(len(item_relevant)) if item_relevant[i]]    
    
    max_r_true = max(rank_item_relevant)
 
    
    p_k = np.zeros(k)    
    for i in range(k):
        p_k[i] = (2**item_relevant[i] - 1) / (2**1 - 1)   
   
    
    P_k = np.zeros(k)    
    P_k[0] = p_k[0]
    for i in range(1, k):
        P_k[i] = p_k[i]
        for j in range(1, i):
            P_k[i] *= (1 - p_k[j]) 
        P_k[i] = P_k[i] / k
   
        
    return sum(P_k)



In [198]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

list=[recommended, boughted, 8]

print("Метрика PR@8 " + str(pr_k(recommended, boughted, 8)))

print("Метрика REC@8  "+ str(rec_k(recommended, boughted, 8)))

print("Метрика AP@8  " + str(ap_k(recommended, boughted, 8)))

print("Метрика NDCG_k  " + str(NDCG_k(recommended, boughted, 8)))

print("Метрика RR_k  " + str(RR_k(recommended, boughted, 8)))

print("Метрика reciprocal_rank@8  " + str(reciprocal_rank(recommended, boughted, 8)))

print("Метрика expected_reciprocal_rank  " + str(expected_reciprocal_rank(recommended, boughted, 8)))



Метрика PR@8 0.5
Метрика REC@8  0.6666666666666666
Метрика AP@8  0.32
Метрика NDCG_k  0.6215894130709897
Метрика RR_k  0.5
Метрика reciprocal_rank@8  0.095
Метрика expected_reciprocal_rank  0.125
